In [ ]:
%pip install transformers gradio

In [ ]:
from transformers import pipeline
import networkx as nx
import gradio as gr

In [ ]:
# parse text-based network
def parse_networks(input_string):
  edges = []
  reversible_edges = []

  for part in input_string.split(', '):
    part = part.strip()
    if '<->' in part:
      a, b = part.split('<->')
      reversible_edges.append((a.strip(), b.strip()))
    elif '->' in part:
      a, b = part.split('->')
      edges.append((a.strip(), b.strip()))

  return edges, reversible_edges

In [ ]:
# build graph using networkx
def build_graph(edges, reversible_edges):
  G = nx.DiGraph()

  G.add_edges_from(edges)
  for a, b in reversible_edges:
    G.add_edge(a, b)
    G.add_edge(b, a)
  return G

In [ ]:
# analyze graph
def analyze_graph(G):
  num_nodes = G.number_of_nodes()
  num_edges = G.number_of_edges()
  is_cyclic = not nx.is_directed_acyclic_graph(G)

  return {"nodes": list(G.nodes),
          "edges": list(G.edges),
          "num_nodes": num_nodes,
          "num_edges":num_edges,
          "is_cyclic": is_cyclic
          }

In [ ]:
# Now, import the pipeline
qa = pipeline("text2text-generation", model="google/flan-t5-base")

In [ ]:
# Define the function that gets responses based on user input
def process_network(query, input_string):
    # Parse the network and build the graph
    edges, reversible = parse_networks(input_string)
    G = build_graph(edges, reversible)
    graph_info = analyze_graph(G)

    # Prepare a prompt for the LLM based on the user's query
    if "variables" in query.lower():
        answer = f"There are {graph_info['num_nodes']} variables (nodes) in the network."
    elif "edges" in query.lower():
        answer = f"The edges in the network are: {graph_info['edges']}."
    elif "cycle" in query.lower() or "cyclic" in query.lower():
        answer = f"Is the network cyclic? {'Yes' if graph_info['is_cyclic'] else 'No'}."
    else:
        # Use the LLM for other specific queries
        prompt = f"Given the network with nodes: {graph_info['nodes']} and edges: {graph_info['edges']}, answer the query: {query}"
        answer = qa(prompt, max_length=128)[0]['generated_text']

    return answer

# Create the Gradio interface
iface = gr.Interface(
    fn=process_network,
    inputs=["text","text"],  # Network description and user query
    outputs="text",  # Answer text
    # live=True,
    title="Biological Network Query System",
    description="Input a network in the form 'A->B, B<->C' and ask questions like 'How many variables?', 'What are the edges?', or 'Is it cyclic?'"
)

# Launch the interface
iface.launch(share=True)
